In [138]:
import cv2
import glob
import time
import numpy as np

start_time = time.time()

# Bilder aus dem Ordner laden
images = glob.glob('images/*.jpeg')

# Erstes Bild als Ausgangspunkt für das Panorama verwenden
panorama = cv2.imread(images[0])

# SIFT-Objekt erstellen
sift = cv2.SIFT_create()

# Loop durch alle Bilder
for image in images[1:]:
    # Bild laden
    current_image = cv2.imread(image)

    # SIFT-Keypoints und -Descriptors für beide Bilder finden
    kp1, des1 = sift.detectAndCompute(panorama, None)
    kp2, des2 = sift.detectAndCompute(current_image, None)

    # Keypoints mit FLANN matchen
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Gültige Matches auswählen
    good_matches = []
    for m, n in matches:
        if m.distance < 0.4*n.distance:
            good_matches.append(m)

    # Keypoints der gültigen Matches bestimmen
    if len(good_matches) > 9:
        panorama_points = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        current_image_points = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        # Homographie-Matrix bestimmen
        H, mask = cv2.findHomography(current_image_points, panorama_points, cv2.RANSAC, 5.0)




        # Neue Größe des Panoramas bestimmen
        panorama_width = panorama.shape[1] + 500
        panorama_height = max(panorama.shape[0], current_image.shape[0])

        # Neues Panorama erstellen
        result = cv2.warpPerspective(current_image, H, (panorama_width, panorama_height))
        result[0:panorama.shape[0], 0:panorama.shape[1]] = panorama

        panorama = result


# Panorama speichern
cv2.imwrite('panorama.jpeg', panorama)

end_time = time.time()
total_time = end_time - start_time
print("Die Rechendauer betrug: ", total_time, "Sekunden")

Die Rechendauer betrug:  17.283375024795532 Sekunden


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()